In [12]:
import cv2
import numpy as np
import os

In [13]:
params_dir = os.getcwd()+'/camera_params/' #Calibration -> Parallel image 
print(params_dir)
# Load camera parameters
ret = np.load(params_dir+'ret.npy')
K = np.load(params_dir+'K.npy')
dist = np.load(params_dir+'dist.npy')
focal_length = (K[0,0] + K[1,1])/2

print(K)

d:\GITHUB\final.image/camera_params/
[[726.47630219   0.         466.25758541]
 [  0.         732.0640605  251.997867  ]
 [  0.           0.           1.        ]]


In [14]:
#Declare stereo matching variables
#This section student has to do fine tunes by yourself for max_disp
win_size = 5
min_disparity = -1
max_disparity = 63
num_disparity = max_disparity - min_disparity #Must be divisible by 16.

In [15]:
def downsampling_image(image, reduction_scale) : # Reduce image resolution Pyr lelel
    for i in range(0, reduction_scale) :
        if len(image.shape) > 2:
            row, col = image.shape[:2]
        else :
            row, col = image.shape
        
        image = cv2.pyrDown(image,dstsize=(col//2,row//2))
    return image

In [16]:
def preprocessimg(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return (img, img_gray)

In [17]:
template = cv2.imread("./images/template/Template-4.png")
template_img,template_gray = preprocessimg(template)
# cv2.imshow("AAA",template_gray)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [19]:
left_cap = cv2.VideoCapture('./video/final_exam/Dataset-1/left_output-1.avi')
right_cap = cv2.VideoCapture('./video/final_exam/Dataset-1/right_output-1.avi')

ret ,left_vid = left_cap.read() # You have to read left and right together for syncing the video frame
ret ,right_vid = right_cap.read() 

h, w = left_vid.shape[:2]

#Get the new optimal camera matrix for better undistortion
new_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(K, dist, (w,h),1, (w,h))

while left_cap.isOpened() and right_cap.isOpened() :
    ret_left ,left_vid = left_cap.read() # You have to read left and right together for syncing the video frame
    ret_right ,right_vid = right_cap.read() 
    
    if ret_left and ret_right :
        left_vid = cv2.cvtColor(left_vid,cv2.COLOR_BGR2GRAY)
        right_vid = cv2.cvtColor(right_vid,cv2.COLOR_BGR2GRAY)

        #Undistort image
        left_vid_undistorted = cv2.undistort(left_vid, K, dist, None, new_camera_matrix)
        right_vid_undistorted = cv2.undistort(right_vid, K, dist, None, new_camera_matrix)

        #Downsampling each image 1 pyramid level in order to improve computational speed
        left_vid_down = downsampling_image(left_vid_undistorted,1)
        right_vi_down = downsampling_image(right_vid_undistorted,1)




        cv2.imshow('Video frame', right_vi_down)
        if cv2.waitKey(33) & 0xFF == ord('q') : # this line control the period between image frame
            break
    else :
        break
cv2.destroyAllWindows()
left_cap.release()
right_cap.release()

AttributeError: 'numpy.ndarray' object has no attribute 'release'